<div style="display: flex; align-items: center;">
    <img src="../figures/Mines Geophysics Black Moon Circle Waves 3.3.png" alt="Example Image" width="10%">
    <div style="margin-left: 10px;">
        <h1>GP100 Alumni Map</h1>
        <h2>Data Analysis</h2>
    </div>
</div>

### Loading the Data

In [11]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
# load data into a dataframe
path1 = '../data/GP_Alumni_Master_Email_Mail_List.csv'
df = pd.read_csv(path1)
df = df.iloc[:,[10, 11, 12, 20, 23]]
df

,Affiliation,PrefClassYear,Degrees,StateOrProvince,Country
0,Alumni,1983.0,"BSc Geop Eng '83, Geop Engineer '99",CO,NaN
1,Alumni,1995.0,BSc Geop Eng '95,CO,NaN
2,Alumni,2013.0,MSc Geop '13,NaN,Bahrain
3,Alumni,1999.0,MSc Geop '99,NaN,United Kingdom
4,Alumni,1982.0,BSc Geop Eng '82,OK,NaN
...,...,...,...,...,...
1919,Alumni,2012.0,BSc Geop Eng '12,CO,NaN
1920,Alumni,1982.0,BSc Geop Eng '82,TX,NaN
1921,Alumni,1981.0,BSc Geop Eng '81,UT,NaN
1922,Alumni,2019.0,BSc Geop Eng '19,CO,NaN


In [9]:
path2 = 
df.to_csv()
df = pd.read_csv(path)
df

,Unnamed: 0,Affiliation,PrefClassYear,Degrees,StateOrProvince,Country
0,0,Alumni,1983.0,"BSc Geop Eng '83, Geop Engineer '99",CO,NaN
1,1,Alumni,1995.0,BSc Geop Eng '95,CO,NaN
2,2,Alumni,2013.0,MSc Geop '13,NaN,Bahrain
3,3,Alumni,1999.0,MSc Geop '99,NaN,United Kingdom
4,4,Alumni,1982.0,BSc Geop Eng '82,OK,NaN
...,...,...,...,...,...,...
1919,1919,Alumni,2012.0,BSc Geop Eng '12,CO,NaN
1920,1920,Alumni,1982.0,BSc Geop Eng '82,TX,NaN
1921,1921,Alumni,1981.0,BSc Geop Eng '81,UT,NaN
1922,1922,Alumni,2019.0,BSc Geop Eng '19,CO,NaN


### Cleaning the Data

In [ ]:
# Delete rows if they are not alumni, and then delete the affiliation column

# Seperate Degrees into BSc, BSc_yr, MSc, MSc_yr, phD, phD_yr

# Remove Canadians
# (if item in StateOrProvince > 2 characters, remove)
# Rename StateOrProvince to just 'State'